# Notebook for finding duplicate data

TODO:

1) For each broker separately - look how much decrease...

2) Phase 2...

Following notebook is to find out how much duplicate requests.


With each request_uuid we have parameters - pickup_timetamp, return_timestamp, broker_contract, driver_age the same..

Duplicates can be spotted as following:

1) Same request_id

2) Different request_id, but close time, same broker and same other parameters (age, source...)



In [1]:
import pandas as pd
import csv
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (14, 6)
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Read data..

fields = ["timestamp", "pickup_timestamp", "return_timestamp", "broker_contract_id", "driver_age", 
          "request_uuid", "source_country_region_id", "pickup_desk_id", "return_desk_id", "rental_days"]
df = pd.read_csv("rate_quote.csv", skipinitialspace=True, usecols=fields)
print(df.shape)





(2997586, 10)


In [3]:
df["timestamp"] = pd.to_datetime(df['timestamp'])
df["pickup_timestamp"] = pd.to_datetime(df['pickup_timestamp'])
df["return_timestamp"] = pd.to_datetime(df['return_timestamp'])


# Merge with broker contract name
fields = ["Contract_ID", "Contract_name", "Broker name"]
contract_df = pd.read_csv("broker_contracts.csv", skipinitialspace=True, usecols=fields)
# This holds contract names and which broker name corresponds to each contract id..
#contract_df = contract_df.set_index("Contract_ID")

print(contract_df.shape)
print(contract_df.dtypes)


contract_df.columns = ["broker_contract_id", "Contract_name", "Broker_name"]
    
#print(df)
    
    
merged_requests = pd.merge(df, contract_df, on=["broker_contract_id"])

print(merged_requests.dtypes)

(27, 3)
Contract_ID       int64
Contract_name    object
Broker name      object
dtype: object
timestamp                   datetime64[ns]
pickup_desk_id                       int64
return_desk_id                       int64
pickup_timestamp            datetime64[ns]
return_timestamp            datetime64[ns]
rental_days                          int64
broker_contract_id                   int64
source_country_region_id             int64
driver_age                           int64
request_uuid                        object
Contract_name                       object
Broker_name                         object
dtype: object


## Phase 1 - Removing duplicates using request_uuid

1) Load in all data

2) Group data by request_uuid, for each find minimal timestamp and set it as aggregate row's timestamp. We can groupby and get first of each (we make the assumption that the data is in chronological order)..

In [4]:
#first_from_each_requestuuid = merged_requests[merged_requests.groupby('request_uuid')['timestamp'].rank() == 1 ]
first_from_each_requestuuid = merged_requests.drop_duplicates("request_uuid")



In [31]:
len(first_from_each_requestuuid)

130915

### How many removed?

Before: 2997586 After: 130915
Decrease of 95.63265240763735%

We see that there are A LOT of duplicates based already only on request_uuid

In [5]:
before_count = len(merged_requests)
after_count =  len(first_from_each_requestuuid)

print("Before:", before_count, "After:", after_count)
print("Decrease of {}%".format((100-(after_count/before_count)*100)))



Before: 2997586 After: 130915
Decrease of 95.63265240763735%


In [6]:
## Saving data frame (without duplicates by requestuuid)

first_from_each_requestuuid.to_csv("rate_quote_1_dup2.csv")

## Phase 2 - Removing extra duplicates - different request_uuid, but other parameters same (nearby timestamp) 

1) Group by broker name

2) Order each group by time

3) Check for orders within 0.5s if other parameters but contract_id / request_uuid are the same

In [7]:
## Group by broker_name

grouped_by_broker = first_from_each_requestuuid.groupby(first_from_each_requestuuid["Broker_name"])



In [9]:
MAX_TIME_DIFFERENCE = 200000

filtered_by_broker = {}
# For every order find all orders in time difference of 1 second.
duplicate_count = 0
for group in grouped_by_broker:
    #ordered_df = group[1].sort_values(by="timestamp")
    previous_row = None
    in_difference = []
    # Group by all other columns now..
    all_other = group[1].groupby(["pickup_timestamp", "return_timestamp", "driver_age", 
          "source_country_region_id", "pickup_desk_id", "return_desk_id", "rental_days"])
    filtered_rows = []
    for group_other in all_other:
        SAME_PARAMETERS = group_other[1].sort_values(by="timestamp")
        # Now from inside group find merge which ones have similar timestamp (Need to set an E - 0.5s?) Consult on this 
        previous_row = None
        for row in SAME_PARAMETERS.itertuples():
            if previous_row:
                # If time difference is enough then add the row.
                if (row[1] - previous_row[1]).microseconds > MAX_TIME_DIFFERENCE:
                    filtered_rows.append(row)
                else:
                    duplicate_count += 1
            else:
                # We add first one no matter what.
                filtered_rows.append(row)
            previous_row = row
    filtered_by_broker[group[0]] = filtered_rows

    
print("Removed {} duplicates".format(duplicate_count))

Removed 51194 duplicates


In [10]:
print("Before removing duplicates")

for group in grouped_by_broker:
    print(group[0], len(group[1]))

print("\nAfter removing duplicates")
    
for k,v in filtered_by_broker.items():
    print(k, len(v))

Before removing duplicates
BSP Auto 1577
Flexible Autos 150
Online Republic 30
RateChain test 55
Skyscanner 8
SupplierWebsite 904
TravelJigsaw 128177
Zuzuche 14

After removing duplicates
BSP Auto 1484
Flexible Autos 132
Online Republic 20
RateChain test 55
Skyscanner 8
SupplierWebsite 868
TravelJigsaw 77144
Zuzuche 10


In [12]:
filtered_df = pd.DataFrame([item for sub in filtered_by_broker.keys() for item in filtered_by_broker[sub]])
print(len(filtered_df))


filtered_df.to_csv("new_rate_quote_1.csv")

79721


In [13]:

print("Before:", before_count, "After:", len(filtered_df))
print("Decrease of {}%".format((100-(len(filtered_df)/before_count)*100)))

Before: 2997586 After: 79721
Decrease of 97.34049331695572%
